In [ ]:
# requirements.py
# Install required packages
!pip install lightgbm scikit-learn pandas numpy


In [ ]:
import os
import gc
import math
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
import lightgbm as lgb

# Set plot style
sns.set_style('whitegrid')
plt.style.use('fivethirtyeight')

RANDOM_STATE = 42

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/mitsui_project/mitsui_project/data/'
train_df = pd.read_csv(f'{file_path}train.csv')
train_labels_df = pd.read_csv(f'{file_path}train_labels.csv')
target_pairs_df = pd.read_csv(f'{file_path}target_pairs.csv')
test_df = pd.read_csv(f'{file_path}test.csv')
df = pd.read_csv(f'{file_path}final_data.csv')
X = pd.read_csv(f'{file_path}X_data.csv')
y = pd.read_csv(f'{file_path}Y_data.csv')

Mounted at /content/drive


In [ ]:
y = y.fillna(method="ffill").fillna(method="bfill")


/tmp/ipython-input-689276991.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y = y.fillna(method="ffill").fillna(method="bfill")


In [ ]:
X

,date_id,LME_AH_Close_logret_lag1,LME_AH_Close_logret_lag2,LME_AH_Close_logret_lag3,LME_AH_Close_logret_lag4,LME_AH_Close_logret_lag5,LME_CA_Close_logret_lag1,LME_CA_Close_logret_lag2,LME_CA_Close_logret_lag3,LME_CA_Close_logret_lag4,...,FX_NOKGBP_ret_mean_5,FX_NOKGBP_ret_std_5,FX_NOKCHF_ret_mean_5,FX_NOKCHF_ret_std_5,FX_ZARCHF_ret_mean_5,FX_ZARCHF_ret_std_5,FX_NOKJPY_ret_mean_5,FX_NOKJPY_ret_std_5,FX_ZARGBP_ret_mean_5,FX_ZARGBP_ret_std_5
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-0.016250,NaN,NaN,NaN,NaN,-0.008083,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.009826,-0.016250,NaN,NaN,NaN,0.005790,-0.008083,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,-0.021337,0.009826,-0.016250,NaN,NaN,-0.009434,0.005790,-0.008083,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,1912,-0.003018,-0.015745,0.015344,0.004024,0.025525,-0.013614,-0.003076,0.000729,0.008264,...,0.000063,0.004137,0.001191,0.007513,0.003139,0.008079,0.000117,0.009221,0.002009,0.008724
1913,1913,-0.012775,-0.003018,-0.015745,0.015344,0.004024,0.008012,-0.013614,-0.003076,0.000729,...,-0.000154,0.004018,0.000247,0.006594,-0.000485,0.006615,-0.001902,0.006545,-0.000887,0.009261
1914,1914,0.008737,-0.012775,-0.003018,-0.015745,0.015344,-0.000420,0.008012,-0.013614,-0.003076,...,0.000889,0.004183,-0.000770,0.006760,0.001100,0.004638,-0.002102,0.006382,0.002757,0.006881
1915,1915,0.000000,0.008737,-0.012775,-0.003018,-0.015745,0.001470,-0.000420,0.008012,-0.013614,...,0.001731,0.004685,0.000170,0.007270,-0.001143,0.003973,0.000074,0.005810,0.000420,0.006842


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
import lightgbm as lgb

# helper
import numpy as np
import pandas as pd


SOLUTION_NULL_FILLER = -999999


def rank_correlation_sharpe_ratio(merged_df: pd.DataFrame) -> float:
    """
    Calculates the rank correlation between predictions and target values,
    and returns its Sharpe ratio (mean / standard deviation).

    :param merged_df: DataFrame containing prediction columns (starting with 'prediction_')
                      and target columns (starting with 'target_')
    :return: Sharpe ratio of the rank correlation
    :raises ZeroDivisionError: If the standard deviation is zero
    """
    prediction_cols = [col for col in merged_df.columns if col.startswith('prediction_')]
    target_cols = [col for col in merged_df.columns if col.startswith('target_')]

    def _compute_rank_correlation(row):
        non_null_targets = [col for col in target_cols if not pd.isnull(row[col])]
        matching_predictions = [col for col in prediction_cols if col.replace('prediction', 'target') in non_null_targets]
        if not non_null_targets:
            raise ValueError('No non-null target values found')
        if row[non_null_targets].std(ddof=0) == 0 or row[matching_predictions].std(ddof=0) == 0:
            raise ZeroDivisionError('Denominator is zero, unable to compute rank correlation.')
        return np.corrcoef(row[matching_predictions].rank(method='average'), row[non_null_targets].rank(method='average'))[0, 1]

    daily_rank_corrs = merged_df.apply(_compute_rank_correlation, axis=1)
    std_dev = daily_rank_corrs.std(ddof=0)
    if std_dev == 0:
        raise ZeroDivisionError('Denominator is zero, unable to compute Sharpe ratio.')
    sharpe_ratio = daily_rank_corrs.mean() / std_dev
    return float(sharpe_ratio)


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Calculates the rank correlation between predictions and target values,
    and returns its Sharpe ratio (mean / standard deviation).
    """
    del solution[row_id_column_name]
    del submission[row_id_column_name]
    assert all(solution.columns == submission.columns)

    submission = submission.rename(columns={col: col.replace('target_', 'prediction_') for col in submission.columns})

    # Not all securities trade on all dates, but solution files cannot contain nulls.
    # The filler value allows us to handle trading halts, holidays, & delistings.
    solution = solution.replace(SOLUTION_NULL_FILLER, None)
    return rank_correlation_sharpe_ratio(pd.concat([solution, submission], axis='columns'))

In [ ]:
from posixpath import split
from lightgbm import early_stopping, log_evaluation

target = "target_0"
y_target = y[target]

tscv = TimeSeriesSplit(n_splits=5)

for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
  X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
  y_train, y_val = y_target.iloc[train_idx], y_target.iloc[val_idx]

  dtrain = lgb.Dataset(X_train, y_train)
  dval = lgb.Dataset(X_val, y_val)

  model = lgb.train({"objective": "regression", "metric": "rmse"}, dtrain, valid_sets=[dtrain, dval], num_boost_round=1000, callbacks=[early_stopping(50), log_evaluation(100)])

  pred_val = model.predict(X_val, num_iteration = model.best_iteration)

  oof_df = pd.DataFrame({"row_id": val_idx, "target": pred_val})

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.043700 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 311215
[LightGBM] [Info] Number of data points in the train set: 322, number of used features: 3241
[LightGBM] [Info] Start training from score 0.000131
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

In [ ]:
submission = pd.DataFrame({"row_id": range(len(y))})
for target in y.columns:
    submission[target] = np.nan  # fill with model preds later


/tmp/ipython-input-510070742.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  submission[target] = np.nan  # fill with model preds later
/tmp/ipython-input-510070742.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  submission[target] = np.nan  # fill with model preds later
/tmp/ipython-input-510070742.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To

In [ ]:
# Suppose 'solution' is train_labels.csv (the real targets)
solution = y.copy()
solution = solution.reset_index().rename(columns={"index": "row_id"})

# Match submission shape
submission = submission.reset_index().rename(columns={"index": "row_id"})

# Use your function
score_value = score(solution, submission, row_id_column_name="row_id")
print("Sharpe ratio score:", score_value)


Sharpe ratio score: nan


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor, early_stopping, log_evaluation

# === STEP 1: Prepare data ===
# Assume X is your feature DataFrame with a 'date_id' column
# Assume y is your labels DataFrame with target_0 ... target_423

X = X.sort_values("date_id").reset_index(drop=True)
y = y.reindex(X.index)

feature_cols = [c for c in X.columns if c not in ["date_id", "time_series_identifier"]]

# === STEP 2: LightGBM params ===
params = {
    "objective": "regression",
    "n_estimators": 2000,
    "learning_rate": 0.05,
    "num_leaves": 64,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "random_state": 42
}

# === STEP 3: CV splitter ===
tscv = TimeSeriesSplit(n_splits=3)

# === STEP 4: Train only on two targets ===
submission = pd.DataFrame({"row_id": range(len(y))})

for target in ["target_0", "target_1"]:
    oof_preds = np.zeros(len(y))

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
        X_train, X_val = X.iloc[train_idx][feature_cols], X.iloc[val_idx][feature_cols]
        y_train, y_val = y.iloc[train_idx][target], y.iloc[val_idx][target]

        model = LGBMRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            callbacks=[early_stopping(50), log_evaluation(100)]
        )

        preds = model.predict(X_val)

        # # === SAFETY: filter out NaN labels ===
        # valid_mask = ~np.isnan(y_val)
        # if valid_mask.sum() > 0:
        #     mse = mean_squared_error(y_val[valid_mask], preds[valid_mask])
        #     rmse = np.sqrt(mse)
        #     print(f"Fold {fold} {target} RMSE: {rmse:.6f}")

        # else:
        #     print(f"Fold {fold} {target}: skipped (all NaN labels)")
    oof_preds[val_idx] = preds
    submission[target] = oof_preds

# === STEP 5: Evaluation Metric ===
SOLUTION_NULL_FILLER = -999999

def rank_correlation_sharpe_ratio(merged_df: pd.DataFrame) -> float:
    prediction_cols = [col for col in merged_df.columns if col.startswith('prediction_')]
    target_cols = [col for col in merged_df.columns if col.startswith('target_')]

    def _compute_rank_correlation(row):
        non_null_targets = [col for col in target_cols if not pd.isnull(row[col])]
        matching_predictions = [col for col in prediction_cols if col.replace('prediction', 'target') in non_null_targets]
        if not non_null_targets:
            return np.nan
        if row[non_null_targets].std(ddof=0) == 0 or row[matching_predictions].std(ddof=0) == 0:
            return np.nan
        return np.corrcoef(row[matching_predictions].rank(method='average'),
                           row[non_null_targets].rank(method='average'))[0, 1]

    daily_rank_corrs = merged_df.apply(_compute_rank_correlation, axis=1)
    daily_rank_corrs = daily_rank_corrs.dropna()
    if len(daily_rank_corrs) == 0:
        return 0.0
    return float(daily_rank_corrs.mean() / daily_rank_corrs.std(ddof=0))

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    solution = solution.copy()
    submission = submission.copy()
    del solution[row_id_column_name]
    del submission[row_id_column_name]
    assert all(solution.columns == submission.columns)
    submission = submission.rename(columns={col: col.replace('target_', 'prediction_') for col in submission.columns})
    solution = solution.replace(SOLUTION_NULL_FILLER, None)
    return rank_correlation_sharpe_ratio(pd.concat([solution, submission], axis=1))

# === STEP 6: Build solution df & evaluate ===
solution = y[["target_0", "target_1"]].copy()
solution = solution.reset_index().rename(columns={"index": "row_id"})
submission = submission.reset_index().rename(columns={"index": "row_id"})

print("Sharpe ratio score:", score(solution, submission, "row_id"))


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032668 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 463056
[LightGBM] [Info] Number of data points in the train set: 480, number of used features: 3240
[LightGBM] [Info] Start training from score 0.000326
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

In [ ]:
import os
import joblib
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor, early_stopping, log_evaluation

# === STEP 4: Train on ALL targets with checkpointing ===
results_dir = "checkpoints"
os.makedirs(results_dir, exist_ok=True)

submission = pd.DataFrame({"row_id": range(len(y))})

for target in y.columns:  # loop through all target_x

    save_path = os.path.join(results_dir, f"{target}_preds.csv")
    model_path = os.path.join(results_dir, f"{target}_model.pkl")

    # Skip if already trained
    if os.path.exists(save_path):
        print(f"Skipping {target}, already done.")
        preds = pd.read_csv(save_path)[target].values
        submission[target] = preds
        continue

    print(f"Training {target} ...")
    oof_preds = np.zeros(len(y))

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
        X_train, X_val = X.iloc[train_idx][feature_cols], X.iloc[val_idx][feature_cols]
        y_train, y_val = y.iloc[train_idx][target], y.iloc[val_idx][target]

        model = LGBMRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            callbacks=[early_stopping(50), log_evaluation(100)]
        )

        preds = model.predict(X_val)
        oof_preds[val_idx] = preds

        # Save last model of this target (optional)
        joblib.dump(model, model_path)

    # Save predictions for this target
    pd.DataFrame({"row_id": range(len(y)), target: oof_preds}).to_csv(save_path, index=False)
    submission[target] = oof_preds
    print(f"Finished {target}, saved predictions.")

# === STEP 5: Merge saved results (in case of resume) ===
for target in y.columns:
    if target not in submission.columns:
        preds = pd.read_csv(os.path.join(results_dir, f"{target}_preds.csv"))
        submission[target] = preds[target].values

# === STEP 6: Build solution df & evaluate ===
solution = y.copy()
solution = solution.reset_index().rename(columns={"index": "row_id"})
submission = submission.reset_index().rename(columns={"index": "row_id"})

print("Sharpe ratio score:", score(solution, submission, "row_id"))


Training date_id ...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.037462 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 463056
[LightGBM] [Info] Number of data points in the train set: 480, number of used features: 3240
[LightGBM] [Info] Start training from score 239.500000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

Exception ignored on calling ctypes callback function: <function _log_callback at 0x7dbf701798a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 287, in _log_callback
    def _log_callback(msg: bytes) -> None:
    
KeyboardInterrupt: 


No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


Exception ignored on calling ctypes callback function: <function _log_callback at 0x7dbf701798a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 287, in _log_callback
    def _log_callback(msg: bytes) -> None:
    
KeyboardInterrupt: 


No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


Exception ignored on calling ctypes callback function: <function _log_callback at 0x7dbf701798a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 287, in _log_callback
    def _log_callback(msg: bytes) -> None:
    
KeyboardInterrupt: 


No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [1]:
import matplotlib.pyplot as plt

nans_cnt = y.isna().sum()

nans_pct = ((nans_cnt/len(y))*100).sort_values(ascending=False)
print(nans_pct.head(20))
print("/noverall nan coverage")
print(nans_pct.describe())

plt.figure(figsize=(12,4))
nans_pct.plot(kind="bar")
plt.ylabel("%nan")
plt.title("nan percentage per target")
plt.show()

NameError: name 'y' is not defined

In [ ]:
# import os
# import joblib
# import pandas as pd

# # Load your test set
# test_data = pd.read_csv("test_data.csv")  # replace with your actual test file path
# X_test = test_data[feature_cols]

# results_dir = "checkpoints"
# submission_test = pd.DataFrame({"row_id": range(len(test_data))})

# # Loop through each target model and predict
# for target in y.columns:  # same target order
#     model_path = os.path.join(results_dir, f"{target}_model.pkl")

#     if not os.path.exists(model_path):
#         print(f"⚠️ Model for {target} not found, skipping...")
#         continue

#     print(f"Loading {target} model and predicting...")
#     model = joblib.load(model_path)

#     preds = model.predict(X_test)
#     submission_test[target] = preds

# # Save final test predictions
# submission_test.to_csv("submission_test.csv", index=False)
# print("✅ Saved test predictions to submission_test.csv")


In [2]:
def sharpe(y_true, y_pred):
    pnl = y_true * y_pred
    return pnl.mean() / pnl.std()

In [3]:
import os
import pandas as pd

results_dir = "checkpoints"

# Load all saved preds
submission = pd.DataFrame()
for file in os.listdir(results_dir):
    if file.endswith("_preds.csv"):
        df = pd.read_csv(os.path.join(results_dir, file))
        if submission.empty:
            submission = df
        else:
            submission = submission.merge(df, on="row_id")

print(submission.head())

   row_id  target_7  target_2  target_1  target_9  target_3  target_0  \
0       0       0.0       0.0       0.0       0.0       0.0       0.0   
1       1       0.0       0.0       0.0       0.0       0.0       0.0   
2       2       0.0       0.0       0.0       0.0       0.0       0.0   
3       3       0.0       0.0       0.0       0.0       0.0       0.0   
4       4       0.0       0.0       0.0       0.0       0.0       0.0   

   date_id  target_4  target_5  target_8  target_6  target_10  
0      0.0       0.0       0.0       0.0       0.0        0.0  
1      0.0       0.0       0.0       0.0       0.0        0.0  
2      0.0       0.0       0.0       0.0       0.0        0.0  
3      0.0       0.0       0.0       0.0       0.0        0.0  
4      0.0       0.0       0.0       0.0       0.0        0.0  
